Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Regression with Regularization

In [16]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
l2_regular_facter = 0.01
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.5*tf.nn.l2_loss(weights) * l2_regular_facter
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [17]:
%%time
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 32.346889
Training accuracy: 11.0%
Validation accuracy: 13.9%
Loss at step 100: 10.440288
Training accuracy: 72.6%
Validation accuracy: 71.0%
Loss at step 200: 6.161615
Training accuracy: 76.3%
Validation accuracy: 74.4%
Loss at step 300: 3.772923
Training accuracy: 79.0%
Validation accuracy: 76.3%
Loss at step 400: 2.423008
Training accuracy: 81.3%
Validation accuracy: 78.1%
Loss at step 500: 1.658149
Training accuracy: 83.0%
Validation accuracy: 79.5%
Loss at step 600: 1.222509
Training accuracy: 84.3%
Validation accuracy: 80.4%
Loss at step 700: 0.972595
Training accuracy: 85.1%
Validation accuracy: 81.2%
Loss at step 800: 0.828229
Training accuracy: 85.5%
Validation accuracy: 81.5%
Test accuracy: 88.6%
CPU times: user 1min 45s, sys: 14.7 s, total: 1min 59s
Wall time: 37.7 s


### One-hidden Layer NN with Regularization

In [25]:
l2_regular_facter = 0.001
hidden_nodes_size = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_nodes_size]))
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  hidden_logits = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
  logits = tf.matmul(tf.nn.softmax(hidden_logits), weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    0.5 * l2_regular_facter * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights)) # l2 regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
            tf.nn.softmax(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), # hidden layer
            weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.softmax(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), # hidden layer 
              weights) + biases)

In [26]:
%%time
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 159.673187
Minibatch accuracy: 12.5%
Validation accuracy: 9.3%
Minibatch loss at step 500: 123.702698
Minibatch accuracy: 57.0%
Validation accuracy: 59.6%
Minibatch loss at step 1000: 96.340256
Minibatch accuracy: 70.3%
Validation accuracy: 72.6%
Minibatch loss at step 1500: 74.880516
Minibatch accuracy: 79.7%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 58.401337
Minibatch accuracy: 84.4%
Validation accuracy: 79.1%
Minibatch loss at step 2500: 45.782482
Minibatch accuracy: 78.9%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 35.856113
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 3500: 27.936251
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 4000: 21.898527
Minibatch accuracy: 87.5%
Validation accuracy: 82.4%
Minibatch loss at step 4500: 17.180107
Minibatch accuracy: 85.2%
Validation accuracy: 82.3%
Minibatch loss at step 5000: 13.464406
Minibatch accuracy: 86.7%
Valida

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [34]:
%%time

l2_regular_facter = 0.001
hidden_nodes_size = 1024
batch_size = 8

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_nodes_size]))
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  hidden_logits = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
  logits = tf.matmul(tf.nn.softmax(hidden_logits), weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    0.5 * l2_regular_facter * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights)) # l2 regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
            tf.nn.softmax(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), # hidden layer
            weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.softmax(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), # hidden layer 
              weights) + biases)
    
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.923870
Minibatch accuracy: 25.0%
Validation accuracy: 8.0%
Minibatch loss at step 500: 16.770042
Minibatch accuracy: 62.5%
Validation accuracy: 61.5%
Minibatch loss at step 1000: 17.067432
Minibatch accuracy: 37.5%
Validation accuracy: 70.6%
Minibatch loss at step 1500: 14.983482
Minibatch accuracy: 87.5%
Validation accuracy: 72.1%
Minibatch loss at step 2000: 15.245594
Minibatch accuracy: 75.0%
Validation accuracy: 75.5%
Minibatch loss at step 2500: 14.729298
Minibatch accuracy: 62.5%
Validation accuracy: 74.2%
Minibatch loss at step 3000: 15.013306
Minibatch accuracy: 62.5%
Validation accuracy: 76.2%
Minibatch loss at step 3500: 14.771928
Minibatch accuracy: 62.5%
Validation accuracy: 76.3%
Minibatch loss at step 4000: 13.453455
Minibatch accuracy: 87.5%
Validation accuracy: 76.7%
Minibatch loss at step 4500: 13.266834
Minibatch accuracy: 87.5%
Validation accuracy: 77.6%
Minibatch loss at step 5000: 12.880273
Minibatch accuracy: 87.5%
Validati

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [35]:
%%time
l2_regular_facter = 0.005
hidden_nodes_size = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_size]))
  hidden_biases = tf.Variable(tf.zeros([hidden_nodes_size]))
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  hidden_logits = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
  logits = tf.matmul(
        tf.nn.dropout(tf.nn.softmax(hidden_logits), 0.5),
        weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    0.5 * l2_regular_facter * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights)) # l2 regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(
            tf.nn.softmax(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), # hidden layer
            weights) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(
        tf.nn.softmax(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), # hidden layer 
              weights) + biases)
    
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 791.490845
Minibatch accuracy: 9.4%
Validation accuracy: 9.9%
Minibatch loss at step 500: 227.606735
Minibatch accuracy: 36.7%
Validation accuracy: 64.2%
Minibatch loss at step 1000: 66.352821
Minibatch accuracy: 42.2%
Validation accuracy: 77.7%
Minibatch loss at step 1500: 19.970970
Minibatch accuracy: 55.5%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 6.776862
Minibatch accuracy: 56.2%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 3.217752
Minibatch accuracy: 46.9%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 2.035551
Minibatch accuracy: 50.8%
Validation accuracy: 82.1%
Minibatch loss at step 3500: 1.710478
Minibatch accuracy: 50.8%
Validation accuracy: 82.8%
Minibatch loss at step 4000: 1.524394
Minibatch accuracy: 59.4%
Validation accuracy: 82.5%
Minibatch loss at step 4500: 1.545213
Minibatch accuracy: 57.0%
Validation accuracy: 82.0%
Minibatch loss at step 5000: 1.483569
Minibatch accuracy: 57.8%
Validation acc

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [39]:
help(tf.train.exponential_decay)

Help on function exponential_decay in module tensorflow.python.training.learning_rate_decay:

exponential_decay(learning_rate, global_step, decay_steps, decay_rate, staircase=False, name=None)
    Applies exponential decay to the learning rate.
    
    When training a model, it is often recommended to lower the learning rate as
    the training progresses.  This function applies an exponential decay function
    to a provided initial learning rate.  It requires a `global_step` value to
    compute the decayed learning rate.  You can just pass a TensorFlow variable
    that you increment at each training step.
    
    The function returns the decayed learning rate.  It is computed as:
    
    ```python
    decayed_learning_rate = learning_rate *
                            decay_rate ^ (global_step / decay_steps)
    ```
    
    If the argument `staircase` is `True`, then `global_step /decay_steps` is an
    integer division and the decayed learning rate follows a staircase function

In [ ]:
%%time
l2_regular_facter = 0.001
hidden_one_size = 1024
hidden_two_size = 100
batch_size = 128
num_steps = 3000

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  hidden_one_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_one_size]))
  hidden_one_biases = tf.Variable(tf.zeros([hidden_one_size]))
    
  hidden_two_weights = tf.Variable(
    tf.truncated_normal([hidden_one_size, hidden_two_size]))
  hidden_two_biases = tf.Variable(tf.zeros([hidden_two_size]))
    
    
  weights = tf.Variable(
    tf.truncated_normal([hidden_two_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  hidden_one_logits = tf.matmul(tf_train_dataset, hidden_one_weights) + hidden_one_biases
  hidden_two_logits = tf.matmul(tf.nn.dropout(tf.nn.softmax(hidden_one_logits), 0.5),
                                hidden_two_weights) + hidden_two_biases
  logits = tf.matmul(
        tf.nn.dropout(tf.nn.softmax(hidden_two_logits), 0.5),
        weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    0.5 * l2_regular_facter * (tf.nn.l2_loss(hidden_one_weights) +
                               tf.nn.l2_loss(hidden_two_weights) +
                               tf.nn.l2_loss(weights)) # l2 regularization
  
  # Optimizer.
#   global_step = tf.Variable(0, trainable=False)
#   learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96)
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  def make_prediction(dset):
    h1 = tf.nn.softmax(tf.matmul(dset, hidden_one_weights) + hidden_one_biases)
    h2 = tf.nn.softmax(tf.matmul(h1, hidden_two_weights) + hidden_two_biases)
    output = tf.nn.softmax(tf.matmul(h2, weights) + biases)
    return output

  valid_prediction = make_prediction(tf_valid_dataset)
  test_prediction = make_prediction(tf_test_dataset)
    

## start tp train
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 177.798203
Minibatch accuracy: 7.8%
Validation accuracy: 10.3%
Minibatch loss at step 500: 138.942398
Minibatch accuracy: 15.6%
Validation accuracy: 22.4%
Minibatch loss at step 1000: 108.702728
Minibatch accuracy: 14.8%
Validation accuracy: 31.6%
Minibatch loss at step 1500: 84.912506
Minibatch accuracy: 22.7%
Validation accuracy: 43.4%
Minibatch loss at step 2000: 66.494896
Minibatch accuracy: 33.6%
Validation accuracy: 54.9%